In [1]:
import nltk

# If haven't done so, go and download
# nltk.download()

In [2]:
import os

trend_files = [f.replace('.json', '') for f in os.listdir('../dataset/trends')]

In [3]:
import pandas as pd

tweet_df = pd.DataFrame([])

for i in trend_files:
    i = pd.read_json('../dataset/trends/'+i+'.json', orient='index').iloc[:100]
    tweet_df = tweet_df.append(i).reset_index().drop('index', axis=1)

In [4]:
tweet_df

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,...,favorited,retweeted,lang,extended_entities,possibly_sensitive,quoted_status_id,quoted_status_id_str,quoted_status,retweeted_status,inferred_text
0,2021-12-03 05:51:02+00:00,1466646136815427590,1466646136815427584,cepet bgt si…. #10YearsOf5SOS,False,"[0, 29]","{'hashtags': [{'text': '10YearsOf5SOS', 'indic...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,in,None,NaN,NaN,NaN,None,None,cepet bgt si…. #10YearsOf5SOS
1,2021-12-03 05:50:54+00:00,1466646103672066053,1466646103672066048,Hallo Guys semoga kalian sehat selalu\n\nYang ...,False,"[0, 279]","{'hashtags': [{'text': 'MoonForJinLA', 'indice...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,in,"{'media': [{'id': 1466646086316015617, 'id_str...",0.0,NaN,NaN,None,None,Hallo Guys semoga kalian sehat selalu\n\nYang ...
2,2021-12-03 05:50:12+00:00,1466645924562694146,1466645924562694144,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,False,"[0, 111]","{'hashtags': [{'text': '10YearsOf5SOS', 'indic...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,in,None,NaN,NaN,NaN,None,None,#10YearsOf5SOS udah 10 tahun ajaa time flies.....
3,2021-12-03 05:49:52+00:00,1466645842148925440,1466645842148925440,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,False,"[0, 52]","{'hashtags': [{'text': 'BackTo2011', 'indices'...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,in,None,0.0,1.466438e+18,1.466438e+18,{'created_at': 'Thu Dec 02 16:05:42 +0000 2021...,None,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...
4,2021-12-03 05:49:15+00:00,1466645689065107456,1466645689065107456,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...",False,"[0, 44]","{'hashtags': [{'text': '10YearsOf5SOS', 'indic...","{'iso_language_code': 'in', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",1.466646e+18,...,False,False,in,"{'media': [{'id': 1466645686250881028, 'id_str...",0.0,NaN,NaN,None,None,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5180,2021-12-03 05:05:16+00:00,1466634616991715332,1466634616991715328,RT @xzhan1005: Xiao Zhan featured in ELLE: Jan...,False,"[0, 107]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,en,"{'media': [{'id': 1466226999559704578, 'id_str...",0.0,NaN,NaN,None,{'created_at': 'Thu Dec 02 02:05:36 +0000 2021...,Xiao Zhan featured in ELLE: January 2021 issue...
5181,2021-12-03 05:05:15+00:00,1466634613292691458,1466634613292691456,RT @gorqiu: xiao's eyeshadow. https://t.co/6Hh...,False,"[0, 53]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",NaN,...,False,False,en,"{'media': [{'id': 1466526770149175302, 'id_str...",0.0,NaN,NaN,None,{'created_at': 'Thu Dec 02 21:56:45 +0000 2021...,xiao's eyeshadow. https://t.co/6HhFnOsQrg
5182,2021-12-03 05:05:04+00:00,1466634569868709889,1466634569868709888,RT @applesncrack: Weibo TV series awards 2021 ...,False,"[0, 143]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,...,False,False,en,None,0.0,NaN,NaN,None,{'created_at': 'Wed Dec 01 05:16:50 +0000 2021...,Weibo TV series awards 2021 tutorial for male ...
5183,2021-12-03 05:04:59+00:00,1466634547693436934,1466634547693436928,RT @chochivee: wisp venti‘s first crush 🍃 #xia...,False,"[0, 91]","{'hashtags': [{'tex

Goal: get sentiment. Therefore get only texts

In [5]:
tweets = pd.DataFrame([])
tweets['raw_text'] = tweet_df['inferred_text']
tweets.head()

,raw_text
0,cepet bgt si…. #10YearsOf5SOS
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h..."


In [6]:
tweets.shape

(5185, 1)

In [7]:
# 1st: clear Twitter-specific terms
import re

def remove_twitter(text, for_gensim = False):
    twitter_stripped_text = re.sub('(@[A-Za-z0-9_]+)', '', text) if for_gensim else re.sub('(@)', '', text)   # Username removal
    twitter_stripped_text = re.sub(r'([http(s)?):\/\/(www\.)?a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*))', '', twitter_stripped_text)   # Link removal
    twitter_stripped_text = re.sub('(\#[A-Za-z0-9_]+)', '', twitter_stripped_text) if for_gensim else re.sub('(\#)', ' ', twitter_stripped_text)  # Hashtag removal, but preserving the word
    twitter_stripped_text = re.sub('([-&])', ' ', twitter_stripped_text)   # dash spacing, accommodating for better representation in spams
    return twitter_stripped_text

tweets['1st_clean_twitter'] = tweets['raw_text'].apply(lambda x: remove_twitter(x))
tweets['1st_clean_twitter_gensim'] = tweets['raw_text'].apply(lambda x: remove_twitter(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...","MYT mohegan sun, 2018 💖 10YearsOf5SOS 5SOS","MYT mohegan sun, 2018 💖"


In [8]:
# 2nd: remove punctuations
import string
punctuation_gensim = re.sub('([.,?!])','', string.punctuation)

def remove_punctuation(text, for_gensim=False):
    clean_text = "".join([i for i in text if i not in punctuation_gensim]) if for_gensim else "".join([i for i in text if i not in string.punctuation])
    clean_text = re.sub('([’])', "'", clean_text)
    return clean_text

tweets['2nd_punctuation'] = tweets['1st_clean_twitter'].apply(lambda x: remove_punctuation(x))
tweets['2nd_punctuation_gensim'] = tweets['1st_clean_twitter_gensim'].apply(lambda x: remove_punctuation(x, for_gensim=True))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...","MYT mohegan sun, 2018 💖 10YearsOf5SOS 5SOS","MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 💖 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018 💖"


In [9]:
import emoji

def remove_emoji(text):
    return re.sub(emoji.get_emoji_regexp(), r"", text)

tweets['3rd_emoji'] = tweets['2nd_punctuation'].apply(lambda x: remove_emoji(x))
tweets['3rd_emoji_gensim'] = tweets['2nd_punctuation_gensim'].apply(lambda x: remove_emoji(x))
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...","MYT mohegan sun, 2018 💖 10YearsOf5SOS 5SOS","MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 💖 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018"


In [10]:
tweets['4th_tokenized'] = tweets['3rd_emoji'].apply(nltk.tokenize.word_tokenize)
tweets['4th_tokenized_gensim'] = tweets['3rd_emoji_gensim'].apply(nltk.tokenize.word_tokenize)
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,"[cepet, bgt, si…, 10YearsOf5SOS]","[cepet, bgt, si…, .]"
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,"[Hallo, Guys, semoga, kalian, sehat, selalu, Y...","[Hallo, Guys, semoga, kalian, sehat, selalu, Y..."
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,"[10YearsOf5SOS, udah, 10, tahun, ajaa, time, f...","[udah, 10, tahun, ajaa, time, flies, .., eh, t..."
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,"[Ashton, mi, nene, BackTo2011, 10YearsOf5SOS, ...","[Ashton, mi, nene]"
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...","MYT mohegan sun, 2018 💖 10YearsOf5SOS 5SOS","MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 💖 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018","[MYT, mohegan, sun, 2018, 10YearsOf5SOS, 5SOS]","[MYT, mohegan, sun, ,, 2018]"


In [11]:
# On the other project that I made I chose stopwords because I was going for bag-of-words approach.
crf = nltk.CRFTagger()
crf.set_model_file('../resources/pos_tag/all_indo_man_tag_corpus_model.crf.tagger')

_ = crf.tag_sents(tweets['4th_tokenized'])

In [12]:
_

[[('cepet', 'FW'), ('bgt', 'FW'), ('si…', 'FW'), ('10YearsOf5SOS', 'CD')],
 [('Hallo', 'NNP'),
  ('Guys', 'NNP'),
  ('semoga', 'VB'),
  ('kalian', 'NN'),
  ('sehat', 'JJ'),
  ('selalu', 'RB'),
  ('Yang', 'NN'),
  ('lagi', 'RB'),
  ('butuh', 'VB'),
  ('atau', 'CC'),
  ('cari', 'VB'),
  ('cari', 'VB'),
  ('kebutuhan', 'NN'),
  ('tinta', 'VB'),
  ('original', 'NN'),
  ('bsa', 'NN'),
  ('diorder', 'NN'),
  ('ditoko', 'NN'),
  ('kmi', 'NN'),
  ('ya', 'NN'),
  ('Jenis', 'NNP'),
  ('Epson', 'NNP'),
  ('Canon', 'NNP'),
  ('HP', 'NNP'),
  ('Pemesanan', 'NNP'),
  ('bsa', 'NN'),
  ('lewat', 'NN'),
  ('Tokopedia', 'NNP'),
  ('SAVE', 'NNP'),
  ('ME', 'NNP'),
  ('MoonForJinLA', 'NNP'),
  ('GanyangSemarang', 'NNP'),
  ('LAHeartsJin', 'NNP'),
  ('10YearsOf5SOS', 'CD'),
  ('5sos2011', 'CD')],
 [('10YearsOf5SOS', 'CD'),
  ('udah', 'NN'),
  ('10', 'CD'),
  ('tahun', 'NN'),
  ('ajaa', 'NN'),
  ('time', 'FW'),
  ('flies', 'FW'),
  ('eh', 'FW'),
  ('tapi', 'CC'),
  ('aku', 'NN'),
  ('baru', 'JJ'),
  ('denge

From here we can see that the POS tagger we use did not tag a bit of the text correctly. Another solution that we have is to load an established Indonesian corpus (from Flair), train the model ourselves, and use it as our POS tagger.

In [13]:
from flair.models import SequenceTagger

pos_tag = SequenceTagger.load('../resources/taggers/indonesian-pos/final-model.pt')

2021-12-06 13:48:05,494 loading file ../resources/taggers/indonesian-pos/final-model.pt


In [14]:
from flair.data import Sentence

Sentence('Budi pergi ke pasar')
# pos_tag.predict(tweets['4th_tokenized'].iloc[0])

Sentence: "Budi pergi ke pasar"   [− Tokens: 4]

In [15]:
tweets['sentence'] = tweets['4th_tokenized'].apply(Sentence)
tweets['sentence'].head()

0    (Token: 1 cepet, Token: 2 bgt, Token: 3 si…, T...
1    (Token: 1 Hallo, Token: 2 Guys, Token: 3 semog...
2    (Token: 1 10YearsOf5SOS, Token: 2 udah, Token:...
3    (Token: 1 Ashton, Token: 2 mi, Token: 3 nene, ...
4    (Token: 1 MYT, Token: 2 mohegan, Token: 3 sun,...
Name: sentence, dtype: object

In [16]:
tweets['sentence']

0       (Token: 1 cepet, Token: 2 bgt, Token: 3 si…, T...
1       (Token: 1 Hallo, Token: 2 Guys, Token: 3 semog...
2       (Token: 1 10YearsOf5SOS, Token: 2 udah, Token:...
3       (Token: 1 Ashton, Token: 2 mi, Token: 3 nene, ...
4       (Token: 1 MYT, Token: 2 mohegan, Token: 3 sun,...
                              ...                        
5180    (Token: 1 Xiao, Token: 2 Zhan, Token: 3 featur...
5181                 (Token: 1 xiaos, Token: 2 eyeshadow)
5182    (Token: 1 Weibo, Token: 2 TV, Token: 3 series,...
5183    (Token: 1 wisp, Token: 2 venti, Token: 3 ‘, To...
5184    (Token: 1 beyond, Token: 2 buying, Token: 3 fr...
Name: sentence, Length: 5185, dtype: object

In [17]:
tweets['sentence'].apply(pos_tag.predict)

0       None
1       None
2       None
3       None
4       None
        ... 
5180    None
5181    None
5182    None
5183    None
5184    None
Name: sentence, Length: 5185, dtype: object

In [18]:
tweets['sentence']

0       (Token: 1 cepet, Token: 2 bgt, Token: 3 si…, T...
1       (Token: 1 Hallo, Token: 2 Guys, Token: 3 semog...
2       (Token: 1 10YearsOf5SOS, Token: 2 udah, Token:...
3       (Token: 1 Ashton, Token: 2 mi, Token: 3 nene, ...
4       (Token: 1 MYT, Token: 2 mohegan, Token: 3 sun,...
                              ...                        
5180    (Token: 1 Xiao, Token: 2 Zhan, Token: 3 featur...
5181                 (Token: 1 xiaos, Token: 2 eyeshadow)
5182    (Token: 1 Weibo, Token: 2 TV, Token: 3 series,...
5183    (Token: 1 wisp, Token: 2 venti, Token: 3 ‘, To...
5184    (Token: 1 beyond, Token: 2 buying, Token: 3 fr...
Name: sentence, Length: 5185, dtype: object

In [19]:
_ = tweets['sentence'].iloc[1]
_.to_tagged_string()

'Hallo <PROPN> Guys <PROPN> semoga <ADV> kalian <PRON> sehat <VERB> selalu <ADV> Yang <PRON> lagi <ADV> butuh <VERB> atau <CCONJ> cari <VERB> cari <NOUN> kebutuhan <NOUN> tinta <NOUN> original <NOUN> bsa <VERB> diorder <ADV> ditoko <VERB> kmi <NOUN> ya <PART> Jenis <NOUN> Epson <PROPN> Canon <PROPN> HP <PROPN> Pemesanan <PROPN> bsa <VERB> lewat <ADP> Tokopedia <PROPN> SAVE <PROPN> ME <PROPN> MoonForJinLA <PROPN> GanyangSemarang <PROPN> LAHeartsJin <PROPN> 10YearsOf5SOS <PROPN> 5sos2011 <PUNCT>'

In [20]:
re.findall(r'(<[^>]*>)', _.to_tagged_string())

['<PROPN>',
 '<PROPN>',
 '<ADV>',
 '<PRON>',
 '<VERB>',
 '<ADV>',
 '<PRON>',
 '<ADV>',
 '<VERB>',
 '<CCONJ>',
 '<VERB>',
 '<NOUN>',
 '<NOUN>',
 '<NOUN>',
 '<NOUN>',
 '<VERB>',
 '<ADV>',
 '<VERB>',
 '<NOUN>',
 '<PART>',
 '<NOUN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<VERB>',
 '<ADP>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PROPN>',
 '<PUNCT>']

In [21]:
# Trial

re.sub('[ ]<', '_', _.to_tagged_string()).replace('>', '')

'Hallo_PROPN Guys_PROPN semoga_ADV kalian_PRON sehat_VERB selalu_ADV Yang_PRON lagi_ADV butuh_VERB atau_CCONJ cari_VERB cari_NOUN kebutuhan_NOUN tinta_NOUN original_NOUN bsa_VERB diorder_ADV ditoko_VERB kmi_NOUN ya_PART Jenis_NOUN Epson_PROPN Canon_PROPN HP_PROPN Pemesanan_PROPN bsa_VERB lewat_ADP Tokopedia_PROPN SAVE_PROPN ME_PROPN MoonForJinLA_PROPN GanyangSemarang_PROPN LAHeartsJin_PROPN 10YearsOf5SOS_PROPN 5sos2011_PUNCT'

In [22]:
sentence_tagged = []

for i in tweets['sentence']:
    sentence_tagged.append(re.sub(r'[ ]<', '_', i.to_tagged_string()).replace('>', ''))

sentence_tagged

['cepet_PROPN bgt_PROPN si…_PROPN 10YearsOf5SOS_PUNCT',
 'Hallo_PROPN Guys_PROPN semoga_ADV kalian_PRON sehat_VERB selalu_ADV Yang_PRON lagi_ADV butuh_VERB atau_CCONJ cari_VERB cari_NOUN kebutuhan_NOUN tinta_NOUN original_NOUN bsa_VERB diorder_ADV ditoko_VERB kmi_NOUN ya_PART Jenis_NOUN Epson_PROPN Canon_PROPN HP_PROPN Pemesanan_PROPN bsa_VERB lewat_ADP Tokopedia_PROPN SAVE_PROPN ME_PROPN MoonForJinLA_PROPN GanyangSemarang_PROPN LAHeartsJin_PROPN 10YearsOf5SOS_PROPN 5sos2011_PUNCT',
 '10YearsOf5SOS_ADV udah_VERB 10_NUM tahun_NOUN ajaa_PROPN time_PROPN flies_PROPN eh_PROPN tapi_CCONJ aku_PRON baru_ADJ dengerin_NOUN lagu_NOUN mereka_PRON pas_ADJ 2014_NUM sih_PART seingetku_ADV',
 'Ashton_PROPN mi_PROPN nene_PROPN BackTo2011_PROPN 10YearsOf5SOS_PROPN 5SOS2011_PUNCT',
 'MYT_PROPN mohegan_PROPN sun_PROPN 2018_NUM 10YearsOf5SOS_PROPN 5SOS_PUNCT',
 'NANGIS_PROPN BGT_PROPN 2011_NUM ENAK_PROPN BANGET_PROPN ALLAHUU_PROPN NGINGETIN_PROPN JAMAN_PROPN JAMAN_PROPN BOPUNG_PUNCT DAH_PROPN BENER_PROPN 

In [23]:
sentence_tagged

['cepet_PROPN bgt_PROPN si…_PROPN 10YearsOf5SOS_PUNCT',
 'Hallo_PROPN Guys_PROPN semoga_ADV kalian_PRON sehat_VERB selalu_ADV Yang_PRON lagi_ADV butuh_VERB atau_CCONJ cari_VERB cari_NOUN kebutuhan_NOUN tinta_NOUN original_NOUN bsa_VERB diorder_ADV ditoko_VERB kmi_NOUN ya_PART Jenis_NOUN Epson_PROPN Canon_PROPN HP_PROPN Pemesanan_PROPN bsa_VERB lewat_ADP Tokopedia_PROPN SAVE_PROPN ME_PROPN MoonForJinLA_PROPN GanyangSemarang_PROPN LAHeartsJin_PROPN 10YearsOf5SOS_PROPN 5sos2011_PUNCT',
 '10YearsOf5SOS_ADV udah_VERB 10_NUM tahun_NOUN ajaa_PROPN time_PROPN flies_PROPN eh_PROPN tapi_CCONJ aku_PRON baru_ADJ dengerin_NOUN lagu_NOUN mereka_PRON pas_ADJ 2014_NUM sih_PART seingetku_ADV',
 'Ashton_PROPN mi_PROPN nene_PROPN BackTo2011_PROPN 10YearsOf5SOS_PROPN 5SOS2011_PUNCT',
 'MYT_PROPN mohegan_PROPN sun_PROPN 2018_NUM 10YearsOf5SOS_PROPN 5SOS_PUNCT',
 'NANGIS_PROPN BGT_PROPN 2011_NUM ENAK_PROPN BANGET_PROPN ALLAHUU_PROPN NGINGETIN_PROPN JAMAN_PROPN JAMAN_PROPN BOPUNG_PUNCT DAH_PROPN BENER_PROPN 

In [24]:
tweets['5_tagged'] = sentence_tagged
tweets['5_tagged'] = tweets['5_tagged'].apply(nltk.tokenize.word_tokenize)

In [25]:
tweets.head()

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim,sentence,5_tagged
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,"[cepet, bgt, si…, 10YearsOf5SOS]","[cepet, bgt, si…, .]","(Token: 1 cepet, Token: 2 bgt, Token: 3 si…, T...","[cepet_PROPN, bgt_PROPN, si…_PROPN, 10YearsOf5..."
1,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,Hallo Guys semoga kalian sehat selalu\n\nYang ...,"[Hallo, Guys, semoga, kalian, sehat, selalu, Y...","[Hallo, Guys, semoga, kalian, sehat, selalu, Y...","(Token: 1 Hallo, Token: 2 Guys, Token: 3 semog...","[Hallo_PROPN, Guys_PROPN, semoga_ADV, kalian_P..."
2,#10YearsOf5SOS udah 10 tahun ajaa time flies.....,10YearsOf5SOS udah 10 tahun ajaa time flies.....,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,10YearsOf5SOS udah 10 tahun ajaa time flies e...,udah 10 tahun ajaa time flies.. eh tapi aku b...,"[10YearsOf5SOS, udah, 10, tahun, ajaa, time, f...","[udah, 10, tahun, ajaa, time, flies, .., eh, t...","(Token: 1 10YearsOf5SOS, Token: 2 udah, Token:...","[10YearsOf5SOS_ADV, udah_VERB, 10_NUM, tahun_N..."
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,"[Ashton, mi, nene, BackTo2011, 10YearsOf5SOS, ...","[Ashton, mi, nene]","(Token: 1 Ashton, Token: 2 mi, Token: 3 nene, ...","[Ashton_PROPN, mi_PROPN, nene_PROPN, BackTo201..."
4,"MYT mohegan sun, 2018 💖 #10YearsOf5SOS #5SOS h...","MYT mohegan sun, 2018 💖 10YearsOf5SOS 5SOS","MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 💖 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018 💖",MYT mohegan sun 2018 10YearsOf5SOS 5SOS,"MYT mohegan sun, 2018","[MYT, mohegan, sun, 2018, 10YearsOf5SOS, 5SOS]","[MYT, mohegan, sun, ,, 2018]","(Token: 1 MYT, Token: 2 mohegan, Token: 3 sun,...","[MYT_PROPN, mohegan_PROPN, sun_PROPN, 2018_NUM..."


In [26]:
tweets.to_json('../dataset/tweet-processed.json', orient='index', default_handler=str)
tweets.to_excel('../dataset/tweet-label.xlsx')

In [27]:
import langdetect
import numpy as np

langs = []
iteration = 1

for i in tweets['1st_clean_twitter_gensim']:
    print(iteration, end='\r')
    try:
        langs.append(langdetect.detect(i))
    except:
        langs.append(np.nan)
    iteration += 1

# tweets['lang'] = tweets['raw_text'].apply(langdetect.detect)
# tweets.head(25)

In [28]:
tweets['lang'] = langs

In [29]:
tweets[tweets['lang'] != 'id']

,raw_text,1st_clean_twitter,1st_clean_twitter_gensim,2nd_punctuation,2nd_punctuation_gensim,3rd_emoji,3rd_emoji_gensim,4th_tokenized,4th_tokenized_gensim,sentence,5_tagged,lang
0,cepet bgt si…. #10YearsOf5SOS,cepet bgt si…. 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,cepet bgt si… 10YearsOf5SOS,cepet bgt si….,"[cepet, bgt, si…, 10YearsOf5SOS]","[cepet, bgt, si…, .]","(Token: 1 cepet, Token: 2 bgt, Token: 3 si…, T...","[cepet_PROPN, bgt_PROPN, si…_PROPN, 10YearsOf5...",ro
3,Ashton mi nene\n#BackTo2011 #10YearsOf5SOS #5...,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,Ashton mi nene\n BackTo2011 10YearsOf5SOS 5...,Ashton mi nene\n,"[Ashton, mi, nene, BackTo2011, 10YearsOf5SOS, ...","[Ashton, mi, nene]","(Token: 1 Ashton, Token: 2 mi, Token: 3 nene, ...","[Ashton_PROPN, mi_PROPN, nene_PROPN, BackTo201...",sq
5,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,NANGIS BGT 2011 ENAK BANGET ALLAHUU NGINGETIN ...,"[NANGIS, BGT, 2011, ENAK, BANGET, ALLAHUU, NGI...","[NANGIS, BGT, 2011, ENAK, BANGET, ALLAHUU, NGI...","(Token: 1 NANGIS, Token: 2 BGT, Token: 3 2011,...","[NANGIS_PROPN, BGT_PROPN, 2011_NUM, ENAK_PROPN...",de
7,Liriknya itu loooo🥺😭\n#10YearsOf5SOS https://t...,Liriknya itu loooo🥺😭\n 10YearsOf5SOS,Liriknya itu loooo🥺😭\n,Liriknya itu loooo🥺😭\n 10YearsOf5SOS,Liriknya itu loooo🥺😭\n,Liriknya itu loooo\n 10YearsOf5SOS,Liriknya itu loooo\n,"[Liriknya, itu, loooo, 10YearsOf5SOS]","[Liriknya, itu, loooo]","(Token: 1 Liriknya, Token: 2 itu, Token: 3 loo...","[Liriknya_NOUN, itu_DET, loooo_VERB, 10YearsOf...",so
9,◜📸◞ #INSTAGRAM — Ryan Fleming te amamos. ❤🎶\n\...,◜📸◞ INSTAGRAM — Ryan Fleming te amamos. ❤🎶\n\...,◜📸◞ — Ryan Fleming te amamos. ❤🎶\n\n,◜📸◞ INSTAGRAM — Ryan Fleming te amamos ❤🎶\n\n...,◜📸◞ — Ryan Fleming te amamos. ❤🎶\n\n,◜◞ INSTAGRAM — Ryan Fleming te amamos \n\n 5S...,◜◞ — Ryan Fleming te amamos. \n\n,"[◜◞, INSTAGRAM, —, Ryan, Fleming, te, amamos, ...","[◜◞, —, Ryan, Fleming, te, amamos, .]","(Token: 1 ◜◞, Token: 2 INSTAGRAM, Token: 3 —, ...","[◜◞_PROPN, INSTAGRAM_PROPN, —_PUNCT, Ryan_PROP...",tl
...,...,...,...,...,...,...,...,...,...,...,...,...
5180,Xiao Zhan featured in ELLE: January 2021 issue...,Xiao Zhan featured in ELLE: January 2021 issue...,Xiao Zhan featured in ELLE: January 2021 issue...,Xiao Zhan featured in ELLE January 2021 issue ...,Xiao Zhan featured in ELLE January 2021 issue ...,Xiao Zhan featured in ELLE January 2021 issue ...,Xiao Zhan featured in ELLE January 2021 issue ...,"[Xiao, Zhan, featured, in, ELLE, January, 2021...","[Xiao, Zhan, featured, in, ELLE, January, 2021...","(Token: 1 Xiao, Token: 2 Zhan, Token: 3 featur...","[Xiao_PROPN, Zhan_PROPN, featured_PROPN, in_PR...",en
5181,xiao's eyeshadow. https://t.co/6HhFnOsQrg,xiao's eyeshadow.,xiao's eyeshadow.,xiaos eyeshadow,xiaos eyeshadow.,xiaos eyeshadow,xiaos eyeshadow.,"[xiaos, eyeshadow]","[xiaos, eyeshadow, .]","(Token: 1 xiaos, Token: 2 eyeshadow)","[xiaos_PROPN, eyeshadow_PUNCT]",so
5182,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,Weibo TV series awards 2021 tutorial for male ...,"[Weibo, TV, series, awards, 2021, tutorial, fo...","[Weibo, TV, series, awards, 2021, tutorial, fo...","(Token: 1 Weibo, Token: 2 TV, Token: 3 series,...","[Weibo_PROPN, TV_PROPN, series_PROPN, awards_P...",en
5183,wisp venti‘s first crush 🍃 #xiaoven #原神 #venti...,wisp venti‘s first crush 🍃 xiaoven 原神 venti...,wisp venti‘s first crush 🍃 #原神,wisp venti‘s first